# Obtendo informações de dados de música do Spotify

## Dados open source

O projeto foi desenvolvido com GCP e Databricks, utilizando Pyspark para manipulação dos arquivos.
Nesse projeto, estamos realizando um ETL, obtendo informações sobre A API da Web do Spotify fornece uma ampla gama de funcionalidades para desenvolvedores.

Até então, não é ideal utilizar spark para a pequena quantidade de dados, o ideial é acima de GB (Giga Bytes), mas é um projeto, visa trazer aprendizados da vida real, com conteúdo técnico.

[Spotify API - Dados de música e reprodução](https://developer.spotify.com/)

[API GET RECOMMENDATIONS](https://developer.spotify.com/documentation/web-api/reference/get-recommendations)

[Documentação](https://developer.spotify.com/documentation/web-api/reference/get-recommendations)

## Configuração e Imports

In [0]:
dbutils.widgets.text("id_client", "your-id-client")
id_client = dbutils.widgets.get("id_client")

dbutils.widgets.text("secret_client", "your-id-secret")
secret_client = dbutils.widgets.get("secret_client")

dbutils.widgets.text("content_type", "application/x-www-form-urlencoded")
content_type = dbutils.widgets.get("content_type")

dbutils.widgets.text("grant_type", "client_credentials")
grant_type = dbutils.widgets.get("grant_type")

In [0]:
import requests
from pyspark.sql.functions import *

# Solicitando token de acesso

In [0]:
# URL da API do Spotify
url = "https://accounts.spotify.com/api/token"

# Dados do corpo da solicitação
data = {
    "grant_type": grant_type,
    "client_id": id_client,
    "client_secret": secret_client
}

response = requests.post(url, data=data)

# Verificar o status da resposta
if response.status_code == 200:
    # A solicitação foi bem-sucedida
    json_response = response.json()
    access_token = json_response["access_token"]
    print("Access Token:", access_token)
else:
    # A solicitação falhou
    print("Falha na solicitação:", response.text)


# Acessando API

In [0]:
# Cabeçalhos da solicitação com o token de acesso
url_api = "https://api.spotify.com/v1/recommendations?seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=rock%2Calternative%2Cpop&seed_tracks=0c6xIDDpzE81m2q797ordA"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": content_type
}

# Fazer a solicitação GET
response_api = requests.get(url_api, headers=headers)

# Verificar o status da resposta
if response_api.status_code == 200:
    # A solicitação foi bem-sucedida
    json_response = response_api.json()
    # Faça o processamento necessário nos dados da resposta
    print("Solicitação com sucesso")
else:
    # A solicitação falhou
    print("Falha na solicitação:", response_api.text)


In [0]:
json_response

# Transformando JSON em DataFrame

In [0]:
df = spark.createDataFrame([json_response])

In [0]:
df.printSchema()

In [0]:
df.display()

# Tratando schema

In [0]:
# Converter as colunas "seeds" e "tracks" em strings JSON
df_json = df.selectExpr("to_json(seeds) AS seeds", "to_json(tracks) AS tracks")

# Transformar as colunas JSON em estruturas aninhadas
df_struct = df_json.select(from_json(df_json.seeds, "array<map<string, int>>").alias("seeds"),
                           from_json(df_json.tracks, "array<map<string, string>>").alias("tracks"))

# Expandir os elementos das colunas "seeds" e "tracks" em linhas separadas
df_seeds = df_struct.select(explode(df_struct.seeds).alias("seeds"))
df_tracks = df_struct.select(explode(df_struct.tracks).alias("tracks"))

df_seeds.display()
df_tracks.display()

In [0]:
seeds_keys = df_seeds.select(explode(map_keys(col("seeds"))).alias("key"))

In [0]:
seeds_keys.display()

In [0]:
seeds_keys = df_seeds.select(explode(map_keys(col("seeds"))).alias("key"))

tracks_keys = df_tracks.select(explode(map_keys(col("tracks"))).alias("key"))

# Criar colunas individuais para cada chave nas colunas "seeds"
for key_row in seeds_keys.collect():
    key = key_row.key
    df_seeds = df_seeds.withColumn(key, col("seeds")[key])

# Criar colunas individuais para cada chave nas colunas "tracks"
for key_row in tracks_keys.collect():
    key = key_row.key
    df_tracks = df_tracks.withColumn(key, col("tracks")[key])

df_seeds.display()
df_tracks.display()

# Excluindo colunas

In [0]:
df_seeds = df_seeds.drop("seeds")
df_tracks = df_tracks.drop("tracks")

# Alterando o nome das colunas de forma recursiva

In [0]:
sufixo = "seeds_"

colunas_existentes = df_seeds.columns

# dicionário para mapear as colunas existentes com os novos nomes
mappings = {coluna: sufixo + coluna for coluna in colunas_existentes}

# renomeando as colunas usando o dicionário de mapeamento
df_seeds_renamed = df_seeds
for coluna, novo_nome in mappings.items():
    df_seeds_renamed = df_seeds_renamed.withColumnRenamed(coluna, novo_nome)

df_seeds_renamed.display()

In [0]:
sufixo = "tracks_"

colunas = df_tracks.columns

df_tracks_renamed = df_tracks
for coluna in colunas:
    novo_nome = sufixo + coluna
    df_tracks_renamed = df_tracks_renamed.withColumnRenamed(coluna, novo_nome)

df_tracks_renamed.display()

# Salvando e criando uma tabela

In [0]:
df_seeds_renamed.write.format("delta").saveAsTable("spotify_recommendations_seeds")

# Criando um Database e salvando

In [0]:
spark.sql("CREATE DATABASE db_spotify")

In [0]:
df_tracks_renamed.write.format("delta").saveAsTable("db_spotify.recommendations_tracks")

# Renomeando tabela

In [0]:
spark.sql("ALTER TABLE default.spotify_recommendations_seeds RENAME TO default.recommendations_seeds")

# Movendo tabela para outro database

In [0]:
%sql
ALTER TABLE default.recommendations_seeds SET DBPROPERTIES (database = 'db_spotify')

# Apagando a tabela e criando no local correto

In [0]:
%sql
DROP TABLE default.recommendations_seeds

In [0]:
df_seeds_renamed.write.format("delta").saveAsTable("db_spotify.recommendations_seeds")

# Realizando listagem de dados pelo Google Storage
[Documentação Storage Google](https://docs.gcp.databricks.com/storage/gcs.html)

# Rascunhos

In [0]:
# # Expandir os elementos da coluna em linhas separadas
# df_exploded = df.select(explode(df.columns).alias("key", "value"))
# df_exploded.display()

# # Converter as colunas "seeds" e "tracks" em literais
# df_exploded = df_exploded.withColumn("key", lit(df_exploded["key"]))
# df_exploded.display()

# # Obter os valores de cada chave em uma coluna separada
# df_transformed = df_exploded.groupBy("key").pivot("key").agg({"value": "first"})
# df_transformed.display()

In [0]:
# # Expandir os elementos das colunas "seeds" e "tracks" em linhas separadas
# df_seeds = df.select(explode(df.seeds).alias("seeds"))
# df_tracks = df.select(explode(df.tracks).alias("tracks"))

# # Obter os valores das chaves das colunas "seeds"
# for key in df.select("seeds").schema["seeds"].dataType.elementType.elementType.keys():
#     df_seeds = df_seeds.withColumn(key, col("seeds")[key])

# # Obter os valores das chaves das colunas "tracks"
# for key in df.select("tracks").schema["tracks"].dataType.elementType.elementType.keys():
#     df_tracks = df_tracks.withColumn(key, col("tracks")[key])

# # Aplicar a função pivot nas colunas "seeds" e "tracks"
# df_seeds_transformed = df_seeds.groupBy().pivot("key1").agg({"key2": "first"})
# df_tracks_transformed = df_tracks.groupBy().pivot("track1").agg({"track2": "first"})

# # Exibir o resultado
# df_seeds_transformed.display()
# df_tracks_transformed.display()

In [0]:
# # Expandir os elementos das colunas "seeds" e "tracks" em linhas separadas
# df_seeds = df.select(explode(df.seeds).alias("seeds"))
# df_tracks = df.select(explode(df.tracks).alias("tracks"))

# # Obter as chaves das colunas "seeds"
# seeds_keys = df.select("seeds").schema["seeds"].valueType.keyType

# # Obter as chaves das colunas "tracks"
# tracks_keys = df.select("tracks").schema["tracks"].valueType.keyType

# # Criar colunas individuais para cada chave nas colunas "seeds"
# for key in seeds_keys:
#     df_seeds = df_seeds.withColumn(key, col("seeds")[key])

# # Criar colunas individuais para cada chave nas colunas "tracks"
# for key in tracks_keys:
#     df_tracks = df_tracks.withColumn(key, col("tracks")[key])

# # Exibir o resultado
# df_seeds.display()
# df_tracks.display()